## Visualizations with dash

### Dynamic multi-input etc
* https://dash.plotly.com/advanced-callbacks
* https://community.plotly.com/t/updating-a-dropdown-menus-contents-dynamically/4920/4
* https://dash-example-index.herokuapp.com/?code=dcc.Dropdown
* https://dash.plotly.com/dash-core-components/dropdown
* https://dash-bootstrap-components.opensource.faculty.ai/examples/iris/

In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [2]:
counts_df = pd.read_csv('dataframes/with_counts/combined_count.csv', index_col=0)

In [3]:
counts_df.head()

,Artist,Song Title,Full Title,Release Date,Year,Month,Day,Pageviews,url,featured_count,...,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,genre,emotion
0,Aaliyah,Are You That Somebody?,Are You That Somebody? by Aaliyah (Ft. Timbaland),1998-05-26,1998.0,5.0,26.0,373960,https://genius.com/Aaliyah-are-you-that-somebo...,1,...,5,4,26,14,1,1,1,1,pop,sadness
1,Aaliyah,Enough Said,Enough Said by Aaliyah (Ft. Drake),2012-08-05,2012.0,8.0,5.0,316333,https://genius.com/Aaliyah-enough-said-lyrics,1,...,5,4,26,14,1,1,1,1,pop,anger
2,Aaliyah,At Your Best (You Are Love),At Your Best (You Are Love) by Aaliyah,1994-08-22,1994.0,8.0,22.0,285549,https://genius.com/Aaliyah-at-your-best-you-ar...,0,...,5,4,26,14,1,1,1,1,pop,sadness
3,Aaliyah,Miss You,Miss You by Aaliyah,2002-11-16,2002.0,11.0,16.0,245608,https://genius.com/Aaliyah-miss-you-lyrics,0,...,5,4,26,14,1,1,1,1,pop,sadness
4,Aaliyah,Age Ain’t Nothing But a Number,Age Ain't Nothing But a Number by Aaliyah,1994-12-06,1994.0,12.0,6.0,207419,https://genius.com/Aaliyah-age-aint-nothing-bu...,0,...,5,4,26,14,1,1,1,1,pop,joy


In [4]:
list(counts_df.columns)

['Artist',
 'Song Title',
 'Full Title',
 'Release Date',
 'Year',
 'Month',
 'Day',
 'Pageviews',
 'url',
 'featured_count',
 'producer_count',
 'writer_count',
 'Song Lyrics',
 'Artist Image',
 'gender',
 'unique_words',
 'total_words',
 'manual_love_count',
 'manual_money_count',
 'manual_violence_count',
 'manual_drugs_count',
 'manual_gendered_count',
 'manual_sadness_count',
 'manual_joy_count',
 'manual_yes_count',
 'manual_no_count',
 'manual_love_word_percent',
 'manual_money_word_percent',
 'manual_violence_word_percent',
 'manual_drugs_word_percent',
 'manual_gendered_word_percent',
 'manual_sadness_word_percent',
 'manual_joy_word_percent',
 'manual_yes_word_percent',
 'manual_no_word_percent',
 'sentiment',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NN',
 'NNS',
 'NNP',
 'NNPS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 'ge

In [5]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [6]:
artists = list(counts_df.Artist.unique())

In [7]:
topics = [
 'manual_love_count',
 'manual_money_count',
 'manual_violence_count',
 'manual_drugs_count',
 'manual_gendered_count',
 'manual_sadness_count',
 'manual_joy_count',
 'manual_yes_count',
 'manual_no_count'
]

In [8]:
# df[df['Artist'] == 'Al Green'].Year.unique()

In [9]:
controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Genre"),
            dcc.Dropdown(
                id='topic-genre-selection',
                value = 'soul',
                options = list(counts_df.genre.unique()),
                multi = True
            )
            ]),
        html.Div(
            [
            dbc.Label("Artist"),
            # dcc.Dropdown(df.Artist.unique(), 'Al Green', id='topic-artist-selection')
            dcc.Dropdown(
                id='topic-artist-selection-dynamic',
                multi = True
            )
            ]),
        html.Div(
            [
            dbc.Label("Manual word topic"),
            dcc.Dropdown(topics, 'manual_love_count', id='topic-selection')
            ]),
        # html.Div(
        #     [
        #     dbc.Label("Year"),
        #     dcc.Dropdown(id='year-selection-dynamic')
        #     ])
    ],
    body=True,
)


app.layout = dbc.Container([
    html.H1(children = 'Genre and Artist manual topic counts by year', style={'textAlign': 'center'}),
    html.P(children="multi artist select result:"),
    html.P(id="multi-artist-dynamic-test"),
    dbc.Row(
        [
            dbc.Col(controls, md=4),
            dbc.Col(dcc.Graph(id='topic-graph-content'), md=8),
        ],
        align="center",
    ),
    # dbc.DropdownMenu(label="Artist", id="artist-selection", children=items),
    # dbc.Select(id="artist-selection", options=items),
    html.Hr(),
], fluid=True)

In [10]:
@app.callback(
    Output('topic-graph-content', 'figure'),
    Input('topic-genre-selection', 'value'),
    Input('topic-artist-selection-dynamic', 'value'),
    Input('topic-selection', 'value'),
    # Input('year-selection-dynamic', 'value')
)
def update_topic_graph(genres, artists, topic):
    genre_df = counts_df[counts_df['genre'].isin(genres)]
    # in case there is only one artist selected
    print(f'genre_df: {genre_df.shape}')
    artists_list = [artist for artist in artists]
    artists_df = counts_df[genre_df['Artist'].isin(artists_list)]
    print(f'artists_df: {artists_df.shape}')
    # dff = df[df.Artist == artist]
    fig = px.bar(artists_df, x='Year', y=topic, color='genre',
                hover_data = ['Artist'])
    return fig

@app.callback(
    Output('multi-artist-dynamic-test', 'children'),
    Input('topic-artist-selection-dynamic', 'value')
)
def test_multi_dynamic(artists):
    return f'please man {artists}'

@app.callback(
    Output('topic-artist-selection-dynamic', 'options'),
    Input('topic-genre-selection', 'value')
)
def set_dynamic_artist_options(genres):
    options = dict()
    for genre in genres:
        genre_artists = list(counts_df[counts_df['genre'] == genre]['Artist'].unique())
        for artist in genre_artists:
            options[artist] = artist
    return options

@app.callback(
    Output('topic-artist-selection-dynamic', 'value'),
    Input('topic-artist-selection-dynamic', 'options')
)
def set_dynamic_artist_value(options):
    # output = [option for option in options]
    # return output
    return options

# @app.callback(
#     Output('year-selection-dynamic', 'options'),
#     Input('artist-selection', 'value')
# )
# def set_dynamic_year_options(selected_artist):
#     years = list( df[df['Artist'] == selected_artist]['Year'].unique() )
#     return [
#         {'label': year, 'value': year} for year in years
#     ]

# @app.callback(
#     Output('year-selection-dynamic', 'value'),
#     Input('year-selection-dynamic', 'options')
# )
# def set_dynamic_year_value(year_options):
#     return year_options[0]['value']

In [11]:
app.run()

In [12]:
counts_df[counts_df['Artist'].isin(['A Tribe Called Quest', 'MF DOOM'])]

,Artist,Song Title,Full Title,Release Date,Year,Month,Day,Pageviews,url,featured_count,...,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,genre,emotion
220,A Tribe Called Quest,Electric Relaxation,Electric Relaxation by A Tribe Called Quest,1993-11-09,1993.0,11.0,9.0,552782,https://genius.com/A-tribe-called-quest-electr...,0,...,5,4,26,14,1,1,1,1,rock,joy
221,A Tribe Called Quest,We the People....,We the People.... by A Tribe Called Quest,2016-11-11,2016.0,11.0,11.0,543961,https://genius.com/A-tribe-called-quest-we-the...,0,...,5,4,26,14,1,1,1,1,rock,anger
222,A Tribe Called Quest,Scenario,Scenario by A Tribe Called Quest (Ft. Leaders ...,1991-09-24,1991.0,9.0,24.0,508665,https://genius.com/A-tribe-called-quest-scenar...,1,...,5,4,26,14,1,1,1,1,rock,anger
223,A Tribe Called Quest,Can I Kick It?,Can I Kick It? by A Tribe Called Quest,1990-04-17,1990.0,4.0,17.0,441701,https://genius.com/A-tribe-called-quest-can-i-...,0,...,5,4,26,14,1,1,1,1,rock,joy
224,A Tribe Called Quest,Check the Rhime,Check the Rhime by A Tribe Called Quest,1991-09-24,1991.0,9.0,24.0,332387,https://genius.com/A-tribe-called-quest-check-...,0,...,5,4,26,14,1,1,1,1,rock,joy
225,A Tribe Called Quest,Buggin’ Out,Buggin' Out by A Tribe Called Quest,1991-09-24,1991.0,9.0,24.0,305617,https://genius.com/A-tribe-called-quest-buggin...,0,...,5,4,26,14,1,1,1,1,rock,anger
226,A Tribe Called Quest,Bonita Applebum,Bonita Applebum by A Tribe Called Quest,1990-02-19,1990.0,2.0,19.0,263762,https://genius.com/A-tribe-called-quest-bonita...,0,...,5,4,26,14,1,1,1,1,rock,joy
227,A Tribe Called Quest,Jazz (We’ve Got),Jazz (We've Got) by A Tribe Called Quest,1991-09-24,1991.0,9.0,24.0,251774,https://genius.com/A-tribe-called-quest-jazz-w...,0,...,5,4,26,14,1,1,1,1,rock,joy
228,A Tribe Called Quest,The Space Program,The Space Program by A Tribe Called Quest,2016-11-11,2016.0,11.0,11.0,238727,https://genius.com/A-tribe-called-quest-the-sp...,0,...,5,4,26,14,1,1,1,1,rock,joy
229,A Tribe Called Quest,Award Tour,Award Tour by A Tribe Called Quest (Ft. Trugoy...,1993-11-09,1993.0,11.0,9.0,231449,https://genius.com/A-tribe-called-quest-award-...,1,...,5,4,26,14,1,1,1,1,rock,joy
